# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x146203a70>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x146764ad0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x147860f80>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x146764ad0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x147860f80>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 747 events
"Project Created" occurred at 2024-03-26T20:23:03.143296+01:00
"Project Created" occurred at 2024-03-27T11:33:36.607694+01:00
"Project Created" occurred at 2024-03-27T11:53:03.839640+01:00
"Project Created" occurred at 2024-03-27T12:03:49.939865+01:00
"Project Created" occurred at 2024-03-27T14:13:52.841768+01:00
"Project Created" occurred at 2024-03-27T14:27:06.956723+01:00
"Project Created" occurred at 2024-03-27T14:34:04.299426+01:00
"Project Created" occurred at 2024-03-27T14:35:22.090325+01:00
"Project Created" occurred at 2024-03-27T14:37:34.203798+01:00
"Project Created" occurred at 2024-03-27T14:38:35.885215+01:00
"Project Created" occurred at 2024-03-27T14:42:05.292073+01:00
"Project Created" occurred at 2024-03-27T15:11:19.048384+01:00
"Project Created" occurred at 2024-03-27T15:50:57.706088+01:00
"Project Created" occurred at 2024-03-27T16:27:20.652316+01:00
"Project Created" occurred at 2024-03-28T15:11:11.182164+01:00
"Project Created" occurred at 2024-03-

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 747 events
Reset projection
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 22 project summaries
Project 60bd8031-a0cd-4cf6-993f-09913847ce49: Project 1
Project da785c95-c6ef-4170-a92f-d5dba5794db8: Project 1
Project 72018aa5-36d0-4c09-b654-94db932f30dc: Project 1
Project 0564c7e2-f43b-4888-8088-b6802f78bc84: Project 1
Project 6a9f5a1e-565d-4401-8047-8db386d39e25: Project 1
Project 9bed1b0f-de4b-4e69-a34e-8fc8f72be182: Project 1
Project f179e7f9-9688-49f3-883f-a452d7fca119: Weixdorf
Project 434f2c16-6599-4ef4-b291-e237fb37a669: 123
Project 0b722172-c451-4aa0-a1b8-6c9c8711618e: 123
Project 4df6fcd6-a798-462e-9456-631a7e88ee32: 123
Project 212b9c5f-d281-44ae-a273-8bb906747190: Project 1
Project 04546e86-31ef-4015-83e5-1b5a12a3c8cf: Project 1
Project 25fea0d0-7c8c-4954-8646-65b18535e8bf: Project 0
Project d68e332e-f8e7-4758-bf2c-02c91c12017e: Project 1
Project 83915c5e-8cea-45d7-a844-c59652d6628f: Project 2
Project ec41f7b9-4c7f-4f02-8f68-beec50cd0177: Project 3
Project d2d20160-9e3d-4642-bcfb-8d9c98e9c829: Project 4
Project a7e38fa7-9e72-42a0-9873-21f68b74